---

## 8.2 Experimentación

### Experimentos para regresión

In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd

import sys
import os
import re
project_path = os.path.abspath('..')
sys.path.insert(1, project_path)

from src.features.preprocessing import custom_features

### Borrador de GridSearch (regresión)

**Candidatos**:
- RandomForest
- GradientBoosting
- KNeighbors
- MLPClassifier

In [3]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import VotingRegressor

In [4]:
from preprocessing import Nothing, CategoriesTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.pipeline import Pipeline


boc_some_values = CountVectorizer(
    tokenizer = CategoriesTokenizer(),
    max_df = 1.0,
    min_df = 0.05
    )


boc_many_values = CountVectorizer(
    tokenizer = CategoriesTokenizer(),
    max_df = 1.0,
    min_df = 2
    )


preprocessing_reg = ColumnTransformer(
    transformers=[
        ('BoC-plat',boc_some_values,'platforms'),
        ('BoC-cat',boc_some_values,'categories'),
        ('BoC-genres',boc_some_values,'genres'),
        ('BoC-tags',boc_some_values,'tags'),

        ('BoC-dev',boc_many_values,'developer'),
        ('BoC-pub',boc_many_values,'publisher'),

        ('OneHotEncoder',OneHotEncoder(handle_unknown='ignore'),['month']),
        ('MinMaxScaler',MinMaxScaler(),['required_age','price','release_date']),
        ('BoxCox',PowerTransformer(method='yeo-johnson'),['achievements','average_playtime','revenue']),
        ('unchanged',Nothing(),['english'])
])

def make_pipeline(metodo):
    pipeline = Pipeline(steps = [
        ('procesamiento',preprocessing_reg),
        ("selector", SelectPercentile(f_classif, percentile=95)),
        ("regressor",metodo),
    ])
    return pipeline

In [5]:
df_train = pd.read_pickle('train.pickle')
df_train = custom_features(df_train)
X_train, X_eval, y_train, y_eval = train_test_split(df_train, df_train['estimated_sells'], test_size=0.3, random_state=0)

In [6]:
df_train.head()

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,tags,achievements,average_playtime,price,short_description,estimated_sells,rating,month,revenue
0,An Aspie Life,2458206.5,1,Bradley Hennessey;Joe Watson,EnderLost Studios,windows,0,Single-player;Steam Achievements,Adventure;Casual;Free to Play;Indie;Simulation,Free to Play;Adventure;Indie,23,0,0.00,One day your roommate Leaves for no reason. Yo...,3914,Mixed,3,0.0
1,GhostControl Inc.,2456814.5,1,bumblebee,Application Systems Heidelberg,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Casual;Indie;Simulation;Strategy,Turn-Based;Indie;Simulation,53,65,10.99,Manage a team of ghosthunters and free London ...,10728,Mixed,6,0.0
2,Deponia,2456145.5,1,Daedalic Entertainment,Daedalic Entertainment,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Indie,Adventure;Point & Click;Comedy,19,217,6.99,"In Deponia, the world has degenerated into a v...",635792,Positive,8,0.0
3,Atlas Reactor,2457665.5,1,Trion Worlds,Trion Worlds,windows,0,Multi-player;Online Multi-Player;Steam Achieve...,Free to Play;Strategy,Free to Play;Multiplayer;Strategy,121,1240,0.00,SEASON 6 NOW LIVE! The battle for Atlas contin...,253864,Positive,10,0.0
4,CHUCHEL,2458184.5,1,Amanita Design,Amanita Design,windows;mac,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Casual;Indie,Adventure;Indie;Casual,7,245,7.99,CHUCHEL is a comedy adventure game from the cr...,49818,Mostly Positive,3,0.0


In [8]:
regresores_exp = [
    GradientBoostingRegressor(n_estimators=25,random_state=0,learning_rate=0.1),  # 0.3
    RandomForestRegressor(n_estimators=250, random_state=0),
    KNeighborsRegressor(metric='euclidean', weights='distance'),
    MLPRegressor(hidden_layer_sizes=(200, 100),learning_rate='invscaling',random_state=0),
    VotingRegressor(estimators=[
        ('GB', GradientBoostingRegressor(n_estimators=25,random_state=0,learning_rate=0.1)),
        ('RF', RandomForestRegressor(n_estimators=250, random_state=0)),
        ('KN', KNeighborsRegressor(metric='euclidean', weights='distance')),
        ('MLP', MLPRegressor(hidden_layer_sizes=(200, 100),learning_rate='invscaling',random_state=0))
    ])
]

In [9]:
for reg in regresores_exp:
    pipe = make_pipeline(reg)
    score = cross_val_score(pipe,df_train, df_train['estimated_sells'],scoring='r2')
    print("CV SCORE {}: {}".format(type(reg).__name__,score))

CV SCORE GradientBoostingRegressor: [ 0.08500979 -0.17471188  0.38804488  0.46433051  0.12070556]


CV SCORE RandomForestRegressor: [0.05073126 0.08331632 0.36686127 0.38865577 0.10049071]


CV SCORE KNeighborsRegressor: [0.00846303 0.18286252 0.05703902 0.15264569 0.02371788]


CV SCORE MLPRegressor: [0.12072098 0.20337998 0.1042858  0.18506948 0.14364934]


CV SCORE VotingRegressor: [0.12531289 0.20366276 0.28189932 0.3381583  0.28153551]


Resumen aplicación de CV a modelos candidatos de regresión:
- CV SCORE GradientBoostingRegressor: [ 0.08500979 -0.17471188  0.38804488  0.46433051  0.12070556]
- CV SCORE RandomForestRegressor: [0.05073126 0.08331632 0.36686127 0.38865577 0.10049071]
- CV SCORE KNeighborsRegressor: [0.00846303 0.18286252 0.05703902 0.15264569 0.02371788]
- CV SCORE MLPRegressor: [0.12072098 0.20337998 0.1042858  0.18506948 0.14364934]
- CV SCORE VotingRegressor: [0.12531289 0.20366276 0.28189932 0.3381583  0.28153551]

| Modelo | Fold 1 | Fold 2 | Fold 3 | Fold 4 | Fold 5 | **Promedio** |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| **GradientBoosting** | 0.08500979  | -0.17471188 | 0.38804488 | 0.46433051 | 0.12070556 | **0.1766** |
| **RandomForest** | 0.05073126 | 0.08331632 | 0.36686127 | 0.38865577 | 0.10049071 | **0.2945** |
| **KNeighborsRegressor** | 0.00846303 | 0.18286252 | 0.05703902 | 0.15264569 | 0.02371788 | **0.0849** |
| **MLP** | 0.12072098 | 0.20337998 | 0.1042858 | 0.18506948 | 0.14364934 | **0.1513** |
| **VotingRegressor** | 0.12531289 | 0.20366276 | 0.28189932 | 0.3381583 | 0.28153551 | **0.2460** |

Como se había ya detectado en la búsqueda de grilla, los modelos (para la métrica R2) muestran una exagerada varianza de resultados. Como ejemplo emblemático, el método Random Forest mostró un alto valor promedio de R2. Sin embargo esto provenía de algunos folds donde los resultados eran bueno, mientras que en otros el valor era casi 0. Al evaluar este método en codalab los resultados obtenidos fueron de 0, lo cual demuestra la nula confianza que podemos tener en un modelo que varía mucho de fold a fold.

Felizmente, un estimador que mostró ser consistentes de fold a fold fue el voto de varios modelos. Su promedio fue menor que para Random Forest, pero se mantenía estable en valores razonables de R2 sin importar la partición. Nos quedamos entonces con este modelo como regresor final, lo cual resultó bien y que en codalab se obtuvo un valor 0.32 de R2, mejorando el baseline de la competencia.